# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [1]:
import numpy as np
import pandas as pd
import optuna
import shap

from sklearn.ensemble import (
    RandomForestClassifier,ExtraTreesClassifier,AdaBoostClassifier,
    StackingClassifier, BaggingClassifier,HistGradientBoostingClassifier,
    GradientBoostingClassifier, VotingClassifier
)
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.preprocessing import LabelEncoder

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

### 데이터 읽어오기


In [2]:
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv("train.csv")
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [3]:
test_data = pd.read_csv("test.csv")
test_data

,Set ID,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,0001be084fbc4aaa9d921f39e595961b,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3J1XF767-1,1,OK,1000.0,NaN,...,195,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
1,0005bbd180064abd99e63f9ed3e1ac80,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4B1XD472-2,1,OK,1000.0,NaN,...,14,NaN,NaN,256,NaN,NaN,1,NaN,NaN,NaN
2,000948934c4140d883d670adcb609584,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3H1XE355-1,1,OK,240.0,NaN,...,98,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
3,000a6bfd02874c6296dc7b2e9c5678a7,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3L1XA128-1,1,OK,1000.0,NaN,...,14,NaN,NaN,0,NaN,NaN,1,NaN,NaN,NaN
4,0018e78ce91343678716e2ea27a51c95,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4A1XA639-1,1,OK,240.0,NaN,...,1,NaN,NaN,215,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000.0,NaN,...,14,NaN,NaN,131,NaN,NaN,1,NaN,NaN,NaN
17357,ffed8923c8a448a98afc641b770be153,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000.0,NaN,...,12,NaN,NaN,279,NaN,NaN,1,NaN,NaN,NaN
17358,fff1e73734da40adbe805359b3efb462,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240.0,NaN,...,4,NaN,NaN,66,NaN,NaN,1,NaN,NaN,NaN
17359,fff8e38bdd09470baf95f71e92075dec,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240.0,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN


In [4]:
# fill2
# dis 1 (835 ,428, 247) -> (458 ,427.9, 243.7) -> (156, 428.0,243.7  ) //// (? ,156,428) -> (835.5 ,428.0 ,243.7) ->(458, 427.9, 243.7)
# dis 2 (305, 1324.2 ,243.5)-> (499.8 ,1324.2,243.5)-> (694  1324.2, 243.5) ////  (? ,694.0(692.8) ,1324.2)-> (305(304.8) , 1324.2, 243.5)-> 499.8 1324.2, 243.5)

In [5]:
# fill2
# dis 1 
#(838.4 ,430.5(430.2), 244.4(244.2)) -> (458.8(9.5,9.2,8.2,0.0,9.0), 431.1(0.5,0.8),244.4(244.2) ) -> (157.0(8.0,7.5) , 430.8(0.5) , 244.4(244.2) )
#(837.9 ,430.2, 244.3(244.4)) -> (460.0(0.5,459.5) , 430.5, 244.3(244.4)) -> (157.5(7.7), 430.5(0.6), 244.3(244.4))
# 838.2, 430.2, 244.3) -> (460.0, 430.5, 244.3) -> (158.0, 430.6,244.3)
# (na , 157.0(6.5)  430.x(1.1) ) -> (838.4(7.9,8.2),  430.x , 244.x)  -> 458.x  430.x  244.x
#                                   (458.7(56개),     430.x    244.x) ->  838.4 430.x   244.x
# (na , 838.4(67개) 430.0  )     -> (157.0)           430.x    244.x) -> 458.x   430.x   244.x

# dis 2 

# 837.7 1323.2 244.x ->   458.8   1322.x  244.x  ->  157.0 1322.x  244.x 
 
# 837.5 2개.. 1323.2 244.6 -> 460.0  1323.2 244.6 ->  158.0  1323.2 244.6

# na  156.x 1322.x            ->837.x ,       1322.x        244.x     ->     458.x    1322.x    244.x
#      156.1                  -> 458.7(30개)   1322.x      244.x      ->     838.1    1322.x   244.x
    
# na  838.1 (65개) 1322.5     -> 156.1        1322.x        244.x      ->     458.x    1322.x   244.x

# na  681.2(1개) 1322.7       -> 681.2          1322.x      225.85?    ->     682.5    1325.8   244.x   abnormal

### 데이터 전처리 열삭제

In [6]:
train_data = train_data.loc[:, train_data.nunique() > 1]
test_data = test_data.loc[:, test_data.nunique() > 1]

In [12]:
columns_train = set(train_data.columns)
columns_test = set(test_data.columns)

# train에는 있고 test에는 없는 컬럼
diff_in_train = columns_train - columns_test
print(diff_in_train)
# test에는 있고 test에는 없는 컬럼
diff_in_test = columns_test - columns_train
print(diff_in_test)

{'target'}
{'Set ID'}


In [13]:
train_data['WorkMode Collect Result_Dam'].value_counts()

 0.000    18972
 7.000    16447
-0.118     1649
 0.012     1347
-0.026      964
 0.007      884
 0.003      125
 0.024      118
Name: WorkMode Collect Result_Dam, dtype: int64

In [14]:
def correct_shifted_rows_Dam(row):
    # B 열이 NaN인 경우 데이터가 밀렸다고 가정
    if row['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'] == row['HEAD Standby Position X Collect Result_Dam']:
        # 데이터를 왼쪽으로 한 칸씩 이동
        row['HEAD Standby Position X Collect Result_Dam'] = row['HEAD Standby Position Y Collect Result_Dam']
        row['HEAD Standby Position Y Collect Result_Dam'] = row['HEAD Standby Position Z Collect Result_Dam']
        row['HEAD Standby Position Z Collect Result_Dam'] = row['Head Clean Position X Collect Result_Dam'] 
        
        row['Head Clean Position X Collect Result_Dam'] = row['Head Clean Position Y Collect Result_Dam']
        row['Head Clean Position Y Collect Result_Dam'] = row['Head Clean Position Z Collect Result_Dam']
        row['Head Clean Position Z Collect Result_Dam'] = row['Head Purge Position X Collect Result_Dam']
        
        row['Head Purge Position X Collect Result_Dam'] = row['Head Purge Position Y Collect Result_Dam']
        row['Head Purge Position Y Collect Result_Dam'] = row['Head Purge Position Z Collect Result_Dam']
        row['Head Purge Position Z Collect Result_Dam'] = row['Head Zero Position X Collect Result_Dam']
        
        
        row['Head Zero Position X Collect Result_Dam'] = row['Head Zero Position Y Collect Result_Dam']
        row['Head Zero Position Y Collect Result_Dam'] = row['Head Zero Position Z Collect Result_Dam']
        row['Head Zero Position Z Collect Result_Dam'] = row['Machine Tact time Collect Result_Dam']
        
        row['Machine Tact time Collect Result_Dam'] = row['PalletID Collect Result_Dam']
        row['PalletID Collect Result_Dam'] = row['Production Qty Collect Result_Dam']
        row['Production Qty Collect Result_Dam'] = row['Receip No Collect Result_Dam']
        row['Receip No Collect Result_Dam'] = row['Stage1 Circle1 Distance Speed Collect Result_Dam']
        
        row['Stage1 Circle1 Distance Speed Collect Result_Dam'] = row['Stage1 Circle2 Distance Speed Collect Result_Dam']
        row['Stage1 Circle2 Distance Speed Collect Result_Dam'] = row['Stage1 Circle3 Distance Speed Collect Result_Dam']
        row['Stage1 Circle3 Distance Speed Collect Result_Dam'] = row['Stage1 Circle4 Distance Speed Collect Result_Dam']
        row['Stage1 Circle4 Distance Speed Collect Result_Dam'] = row['Stage1 Line1 Distance Speed Collect Result_Dam']
        
        row['Stage1 Line1 Distance Speed Collect Result_Dam'] = row['Stage1 Line2 Distance Speed Collect Result_Dam']
        row['Stage1 Line2 Distance Speed Collect Result_Dam'] = row['Stage1 Line3 Distance Speed Collect Result_Dam']
        row['Stage1 Line3 Distance Speed Collect Result_Dam'] = row['Stage1 Line4 Distance Speed Collect Result_Dam']
        row['Stage1 Line4 Distance Speed Collect Result_Dam'] = row['Stage2 Circle1 Distance Speed Collect Result_Dam']
        
        
        row['Stage2 Circle1 Distance Speed Collect Result_Dam'] = row['Stage2 Circle2 Distance Speed Collect Result_Dam']
        row['Stage2 Circle2 Distance Speed Collect Result_Dam'] = row['Stage2 Circle3 Distance Speed Collect Result_Dam']
        row['Stage2 Circle3 Distance Speed Collect Result_Dam'] = row['Stage2 Circle4 Distance Speed Collect Result_Dam']
        row['Stage2 Circle4 Distance Speed Collect Result_Dam'] = row['Stage2 Line1 Distance Speed Collect Result_Dam']
        
        row['Stage2 Line1 Distance Speed Collect Result_Dam'] = row['Stage2 Line2 Distance Speed Collect Result_Dam']
        row['Stage2 Line2 Distance Speed Collect Result_Dam'] = row['Stage2 Line3 Distance Speed Collect Result_Dam']
        row['Stage2 Line3 Distance Speed Collect Result_Dam'] = row['Stage2 Line4 Distance Speed Collect Result_Dam']
        row['Stage2 Line4 Distance Speed Collect Result_Dam'] = row['Stage3 Circle1 Distance Speed Collect Result_Dam']
        
        
        
        row['Stage3 Circle1 Distance Speed Collect Result_Dam'] = row['Stage3 Circle2 Distance Speed Collect Result_Dam']
        row['Stage3 Circle2 Distance Speed Collect Result_Dam'] = row['Stage3 Circle3 Distance Speed Collect Result_Dam']
        row['Stage3 Circle3 Distance Speed Collect Result_Dam'] = row['Stage3 Circle4 Distance Speed Collect Result_Dam']
        row['Stage3 Circle4 Distance Speed Collect Result_Dam'] = row['Stage3 Line1 Distance Speed Collect Result_Dam']
        
        row['Stage3 Line1 Distance Speed Collect Result_Dam'] = row['Stage3 Line2 Distance Speed Collect Result_Dam']
        row['Stage3 Line2 Distance Speed Collect Result_Dam'] = row['Stage3 Line3 Distance Speed Collect Result_Dam']
        row['Stage3 Line3 Distance Speed Collect Result_Dam'] = row['Stage3 Line4 Distance Speed Collect Result_Dam']
        row['Stage3 Line4 Distance Speed Collect Result_Dam'] = row['THICKNESS 1 Collect Result_Dam']
        
        row['THICKNESS 1 Collect Result_Dam'] = row['THICKNESS 2 Collect Result_Dam']
        row['THICKNESS 2 Collect Result_Dam'] = row['THICKNESS 3 Collect Result_Dam']
        row['THICKNESS 3 Collect Result_Dam'] = row['WorkMode Collect Result_Dam']
        row['WorkMode Collect Result_Dam'] = 7
        
        
        # 마지막 열은 None 또는 NaN으로 설정
    return row

In [15]:
def correct_shifted_rows_Fill1(row):
    # B 열이 NaN인 경우 데이터가 밀렸다고 가정
    if row['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'] == row['HEAD Standby Position X Collect Result_Fill1']:
        # 데이터를 왼쪽으로 한 칸씩 이동
        row['HEAD Standby Position X Collect Result_Fill1'] = row['HEAD Standby Position Y Collect Result_Fill1']
        row['HEAD Standby Position Y Collect Result_Fill1'] = row['HEAD Standby Position Z Collect Result_Fill1']
        row['HEAD Standby Position Z Collect Result_Fill1'] = row['Head Clean Position X Collect Result_Fill1'] 
        
        row['Head Clean Position X Collect Result_Fill1'] = row['Head Clean Position Y Collect Result_Fill1']
        row['Head Clean Position Y Collect Result_Fill1'] = row['Head Clean Position Z Collect Result_Fill1']
        row['Head Clean Position Z Collect Result_Fill1'] = row['Head Purge Position X Collect Result_Fill1']
        
        row['Head Purge Position X Collect Result_Fill1'] = row['Head Purge Position Y Collect Result_Fill1']
        row['Head Purge Position Y Collect Result_Fill1'] = row['Head Purge Position Z Collect Result_Fill1']
        row['Head Purge Position Z Collect Result_Fill1'] = row['Machine Tact time Collect Result_Fill1']
        
        row['Machine Tact time Collect Result_Fill1'] = row['PalletID Collect Result_Fill1']
        row['PalletID Collect Result_Fill1'] = row['Production Qty Collect Result_Fill1']
        row['Production Qty Collect Result_Fill1'] = row['Receip No Collect Result_Fill1']
        row['Receip No Collect Result_Fill1'] = row['WorkMode Collect Result_Fill1']
        row['WorkMode Collect Result_Fill1'] = 7
        
        
        # 마지막 열은 None 또는 NaN으로 설정
    return row

In [16]:
def correct_shifted_rows_Fill2(row):
    # B 열이 NaN인 경우 데이터가 밀렸다고 가정
    if row['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'] == row['HEAD Standby Position X Collect Result_Fill2']:
        # 데이터를 왼쪽으로 한 칸씩 이동
        row['HEAD Standby Position X Collect Result_Fill2'] = row['HEAD Standby Position Y Collect Result_Fill2']
        row['HEAD Standby Position Y Collect Result_Fill2'] = row['HEAD Standby Position Z Collect Result_Fill2']
        row['HEAD Standby Position Z Collect Result_Fill2'] = row['Head Clean Position X Collect Result_Fill2'] 
        
        row['Head Clean Position X Collect Result_Fill2'] = row['Head Clean Position Y Collect Result_Fill2']
        row['Head Clean Position Y Collect Result_Fill2'] = row['Head Clean Position Z Collect Result_Fill2']
        row['Head Clean Position Z Collect Result_Fill2'] = row['Head Purge Position X Collect Result_Fill2']
        
        row['Head Purge Position X Collect Result_Fill2'] = row['Head Purge Position Y Collect Result_Fill2']
        row['Head Purge Position Y Collect Result_Fill2'] = row['Head Purge Position Z Collect Result_Fill2']
        row['Head Purge Position Z Collect Result_Fill2'] = row['Machine Tact time Collect Result_Fill2']
        
        row['Machine Tact time Collect Result_Fill2'] = row['PalletID Collect Result_Fill2']
        row['PalletID Collect Result_Fill2'] = row['Production Qty Collect Result_Fill2']
        row['Production Qty Collect Result_Fill2'] = row['Receip No Collect Result_Fill2']
        row['Receip No Collect Result_Fill2'] = row['WorkMode Collect Result_Fill2']
        row['WorkMode Collect Result_Fill2'] = 0
        
        
        # 마지막 열은 None 또는 NaN으로 설정
    return row

In [18]:
train_data = train_data.apply(correct_shifted_rows_Dam, axis=1)
test_data = test_data.apply(correct_shifted_rows_Dam, axis=1)

In [19]:
train_data = train_data.apply(correct_shifted_rows_Fill1, axis=1)
test_data = test_data.apply(correct_shifted_rows_Fill1, axis=1)

In [20]:
train_data = train_data.apply(correct_shifted_rows_Fill2, axis=1)
test_data = test_data.apply(correct_shifted_rows_Fill2, axis=1)

In [21]:
drop_cols = []
for column in test_data.columns:
    if test_data[column].isnull().sum() ==17361:
        drop_cols.append(column)
test_data = test_data.drop(drop_cols, axis=1)

In [22]:
drop_cols = []
for column in train_data.columns:
    if train_data[column].isnull().sum() ==40506:
        drop_cols.append(column)
train_data = train_data.drop(drop_cols, axis=1)

In [26]:
train_data.iloc[37520][['DISCHARGED SPEED OF RESIN Collect Result_Fill1', 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1']]

DISCHARGED SPEED OF RESIN Collect Result_Fill1           0
DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1    0
Name: 37520, dtype: object

In [27]:
train_data = train_data.drop([37520])

# 머신 댐 공정시간이 999.9 로 오류 테스트에는 없음
#train_data = train_data.drop([27445])

In [28]:
columns_train = set(train_data.columns)
columns_test = set(test_data.columns)

# train에는 있고 test에는 없는 컬럼
diff_in_train = columns_train - columns_test
print(diff_in_train)
# test에는 있고 test에는 없는 컬럼
diff_in_test = columns_test - columns_train
print(diff_in_test)

{'target'}
{'Set ID'}


In [29]:
# Head Purge Position X Collect Result_Dam (resin drain_ 좌표) 가 118.85 인 33개는 절대 abnormal 이 아니다?

### 결측치 처리

In [30]:
missing_values = train_data.isnull().sum()

# 결측치가 있는 열만 선택
missing_columns = missing_values[missing_values >0]
missing_columns

HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam      12765
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1    12765
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2    12765
dtype: int64

In [31]:
missing_values = test_data.isnull().sum()

# 결측치가 있는 열만 선택
missing_columns = missing_values[missing_values >0]
missing_columns 

HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam      5468
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1    5468
HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2    5468
dtype: int64

In [32]:
train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'].value_counts()

OK       11293
549       7271
162.4     3580
550       2398
550.3     1909
549.5     1263
548.5       26
Name: HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam, dtype: int64

In [780]:
# 결측치 있는 dam 열 에서 OK 데이터들도 결측치화
train_data.loc[train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] =='OK', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] = np.nan
test_data.loc[test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] =='OK', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] = np.nan

In [781]:
train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] = pd.to_numeric(
    train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'], errors='coerce'
)
test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] = pd.to_numeric(
    test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'], errors='coerce'
)

In [782]:
def fill_na_based_on_stage2(stage2_value):
    if stage2_value >= 500:
        return stage2_value - 388.9
    elif stage2_value <= 170:
        return stage2_value + 388.9
    else:
        return np.nan  
# 결측치 채우기
train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] = train_data.apply(
    lambda row: fill_na_based_on_stage2(row['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam'])
    if np.isnan(row['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']) 
    else row['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'], 
    axis=1
)

In [783]:
test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] = test_data.apply(
    lambda row: fill_na_based_on_stage2(row['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam'])
    if np.isnan(row['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam']) else row['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'], 
    axis=1
)

In [784]:
train_data.loc[train_data['Equipment_Fill2'] == 'Fill2 dispenser #1',
               'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = '835.5'
train_data.loc[train_data['Equipment_Fill2'] == 'Fill2 dispenser #2',
               'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = '305'

test_data.loc[test_data['Equipment_Fill2'] == 'Fill2 dispenser #1',
              'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = '835.5'
test_data.loc[test_data['Equipment_Fill2'] == 'Fill2 dispenser #2',
              'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = '305'

In [785]:
# 결측치 있는 fill1 열 에서 OK 데이터들도 결측치화
train_data.loc[train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] =='OK', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] = np.nan
test_data.loc[test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] =='OK', 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] = np.nan

In [786]:
train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] = pd.to_numeric(
    train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'], errors='coerce'
)
test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] = pd.to_numeric(
    test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'], errors='coerce'
)

In [787]:
def fill_na_based_on_stage2(stage2_value):
    if stage2_value == 458.7:
        return 157.0
    elif stage2_value <= 170:
        return stage2_value + 388.9
    elif stage2_value >= 800:
        return stage2_value -681.4
    else:
        return np.nan  # 조건에 맞지 않으면 NaN 그대로 둠

# 결측치 채우기
train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] = train_data.apply(
    lambda row: fill_na_based_on_stage2(row['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1'])
    if np.isnan(row['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']) else row['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'], 
    axis=1
)

test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] = test_data.apply(
    lambda row: fill_na_based_on_stage2(row['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1'])
    if np.isnan(row['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1']) else row['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'], 
    axis=1
)

In [788]:
# 결측치 있는 열들 값 수치화
train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = pd.to_numeric(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'], errors='coerce')
test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] = pd.to_numeric(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'], errors='coerce')

In [789]:
# train_data['HEAD_NORMAL_X_DIFF_STAGE1_2_DAM'] = abs(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] - 
#                                 train_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam'])

# train_data['HEAD_NORMAL_X_DIFF_STAGE2_3_DAM'] = abs(train_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam'] - 
#                                 train_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam'])

# train_data['HEAD_NORMAL_X_DIFF_STAGE1_3_DAM'] = abs(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] - 
#                                 train_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam'])


# train_data['HEAD_NORMAL_Y_DIFF_STAGE1_2_DAM'] = abs(train_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam'] - 
#                                 train_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam'])

# train_data['HEAD_NORMAL_Y_DIFF_STAGE2_3_DAM'] = abs(train_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam'] - 
#                                 train_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam'])

# train_data['HEAD_NORMAL_Y_DIFF_STAGE1_3_DAM'] = abs(train_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam'] - 
#                                 train_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam'])



# train_data['HEAD_NORMAL_Z_DIFF_STAGE1_2_DAM'] = abs(train_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam'] - 
#                                 train_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'])
# train_data['HEAD_NORMAL_Z_DIFF_STAGE2_3_DAM'] = abs(train_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'] - 
#                                 train_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'])
# train_data['HEAD_NORMAL_Z_DIFF_STAGE1_3_DAM'] = abs(train_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam'] - 
#                                 train_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'])


In [790]:
# test_data['HEAD_NORMAL_X_DIFF_STAGE1_2_DAM'] = abs(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] - 
#                                 test_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam'])

# test_data['HEAD_NORMAL_X_DIFF_STAGE2_3_DAM'] = abs(test_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam'] - 
#                                 test_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam'])

# test_data['HEAD_NORMAL_X_DIFF_STAGE1_3_DAM'] = abs(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam'] - 
#                                 test_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam'])


# test_data['HEAD_NORMAL_Y_DIFF_STAGE1_2_DAM'] = abs(test_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam'] - 
#                                 test_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam'])

# test_data['HEAD_NORMAL_Y_DIFF_STAGE2_3_DAM'] = abs(test_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam'] - 
#                                 test_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam'])

# test_data['HEAD_NORMAL_Y_DIFF_STAGE1_3_DAM'] = abs(test_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam'] - 
#                                 test_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam'])



# test_data['HEAD_NORMAL_Z_DIFF_STAGE1_2_DAM'] = abs(test_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam'] - 
#                                 test_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'])
# test_data['HEAD_NORMAL_Z_DIFF_STAGE2_3_DAM'] = abs(test_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam'] - 
#                                 test_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'])
# test_data['HEAD_NORMAL_Z_DIFF_STAGE1_3_DAM'] = abs(test_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam'] - 
#                                 test_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam'])


In [791]:
train_data['DISPENSE_VOLUME_DIFF_STAGE1_2_DAM'] = abs(train_data['Dispense Volume(Stage1) Collect Result_Dam'] - 
                                train_data['Dispense Volume(Stage2) Collect Result_Dam'])

train_data['DISPENSE_VOLUME_DIFF_STAGE2_3_DAM'] = abs(train_data['Dispense Volume(Stage2) Collect Result_Dam'] - 
                                train_data['Dispense Volume(Stage3) Collect Result_Dam'])

train_data['DISPENSE_VOLUME_DIFF_STAGE1_3_DAM'] = abs(train_data['Dispense Volume(Stage1) Collect Result_Dam'] - 
                                train_data['Dispense Volume(Stage3) Collect Result_Dam'])

test_data['DISPENSE_VOLUME_DIFF_STAGE1_2_DAM'] = abs(test_data['Dispense Volume(Stage1) Collect Result_Dam'] - 
                                test_data['Dispense Volume(Stage2) Collect Result_Dam'])

test_data['DISPENSE_VOLUME_DIFF_STAGE2_3_DAM'] = abs(test_data['Dispense Volume(Stage2) Collect Result_Dam'] - 
                                test_data['Dispense Volume(Stage3) Collect Result_Dam'])

test_data['DISPENSE_VOLUME_DIFF_STAGE1_3_DAM'] = abs(test_data['Dispense Volume(Stage1) Collect Result_Dam'] - 
                                test_data['Dispense Volume(Stage3) Collect Result_Dam'])


/tmp/ipykernel_149/3671528853.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['DISPENSE_VOLUME_DIFF_STAGE1_2_DAM'] = abs(train_data['Dispense Volume(Stage1) Collect Result_Dam'] -
/tmp/ipykernel_149/3671528853.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['DISPENSE_VOLUME_DIFF_STAGE2_3_DAM'] = abs(train_data['Dispense Volume(Stage2) Collect Result_Dam'] -
/tmp/ipykernel_149/3671528853.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` ma

In [792]:
# train_data['HEAD_NORMAL_X_DIFF_STAGE1_2_Fill1'] = abs(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] - 
#                                 train_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1'])

# train_data['HEAD_NORMAL_X_DIFF_STAGE2_3_Fill1'] = abs(train_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1'] - 
#                                 train_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'])

# train_data['HEAD_NORMAL_X_DIFF_STAGE1_3_Fill1'] = abs(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] - 
#                                 train_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'])


# train_data['HEAD_NORMAL_Y_DIFF_STAGE1_2_Fill1'] = abs(train_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1'] - 
#                                 train_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1'])

# train_data['HEAD_NORMAL_Y_DIFF_STAGE2_3_Fill1'] = abs(train_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1'] - 
#                                 train_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1'])

# train_data['HEAD_NORMAL_Y_DIFF_STAGE1_3_Fill1'] = abs(train_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1'] - 
#                                 train_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1'])



# train_data['HEAD_NORMAL_Z_DIFF_STAGE1_2_Fill1'] = abs(train_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1'] - 
#                                 train_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'])

# train_data['HEAD_NORMAL_Z_DIFF_STAGE2_3_Fill1'] = abs(train_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'] - 
#                                 train_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'])

# train_data['HEAD_NORMAL_Z_DIFF_STAGE1_3_Fill1'] = abs(train_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1'] - 
#                                 train_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'])


In [793]:
train_data['DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1'] = abs(train_data['Dispense Volume(Stage1) Collect Result_Fill1'] - 
                                train_data['Dispense Volume(Stage2) Collect Result_Fill1'])

train_data['DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1'] = abs(train_data['Dispense Volume(Stage2) Collect Result_Fill1'] - 
                                train_data['Dispense Volume(Stage3) Collect Result_Fill1'])

train_data['DISPENSE_VOLUME_DIFF_STAGE1_3_Fill1'] = abs(train_data['Dispense Volume(Stage1) Collect Result_Fill1'] - 
                                train_data['Dispense Volume(Stage3) Collect Result_Fill1'])

test_data['DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1'] = abs(test_data['Dispense Volume(Stage1) Collect Result_Fill1'] - 
                                test_data['Dispense Volume(Stage2) Collect Result_Fill1'])

test_data['DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1'] = abs(test_data['Dispense Volume(Stage2) Collect Result_Fill1'] - 
                                test_data['Dispense Volume(Stage3) Collect Result_Fill1'])

test_data['DISPENSE_VOLUME_DIFF_STAGE1_3_Fill1'] = abs(test_data['Dispense Volume(Stage1) Collect Result_Fill1'] - 
                                test_data['Dispense Volume(Stage3) Collect Result_Fill1'])


/tmp/ipykernel_149/536359753.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1'] = abs(train_data['Dispense Volume(Stage1) Collect Result_Fill1'] -
/tmp/ipykernel_149/536359753.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1'] = abs(train_data['Dispense Volume(Stage2) Collect Result_Fill1'] -
/tmp/ipykernel_149/536359753.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inser

In [794]:
# test_data['HEAD_NORMAL_X_DIFF_STAGE1_2_Fill1'] = abs(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] - 
#                                 test_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1'])

# test_data['HEAD_NORMAL_X_DIFF_STAGE2_3_Fill1'] = abs(test_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1'] - 
#                                 test_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'])

# test_data['HEAD_NORMAL_X_DIFF_STAGE1_3_Fill1'] = abs(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1'] - 
#                                 test_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1'])


# test_data['HEAD_NORMAL_Y_DIFF_STAGE1_2_Fill1'] = abs(test_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1'] - 
#                                 test_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1'])

# test_data['HEAD_NORMAL_Y_DIFF_STAGE2_3_Fill1'] = abs(test_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1'] - 
#                                 test_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1'])

# test_data['HEAD_NORMAL_Y_DIFF_STAGE1_3_Fill1'] = abs(test_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1'] - 
#                                 test_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1'])

# test_data['HEAD_NORMAL_Z_DIFF_STAGE1_2_Fill1'] = abs(test_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1'] - 
#                                 test_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'])

# test_data['HEAD_NORMAL_Z_DIFF_STAGE2_3_Fill1'] = abs(test_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1'] - 
#                                 test_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'])

# test_data['HEAD_NORMAL_Z_DIFF_STAGE1_3_Fill1'] = abs(test_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1'] - 
#                                 test_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1'])


In [795]:
# train_data['HEAD_NORMAL_X_DIFF_STAGE1_2_Fill2'] = abs(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] - 
#                                 train_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'])

# train_data['HEAD_NORMAL_X_DIFF_STAGE2_3_Fill2'] = abs(train_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'] - 
#                                 train_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'])

# train_data['HEAD_NORMAL_X_DIFF_STAGE1_3_Fill2'] = abs(train_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] - 
#                                 train_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'])


# train_data['HEAD_NORMAL_Y_DIFF_STAGE1_2_Fill2'] = abs(train_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2'] - 
#                                 train_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2'])

# train_data['HEAD_NORMAL_Y_DIFF_STAGE2_3_Fill2'] = abs(train_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2'] - 
#                                 train_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2'])

# train_data['HEAD_NORMAL_Y_DIFF_STAGE1_3_Fill2'] = abs(train_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2'] - 
#                                 train_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2'])



# train_data['HEAD_NORMAL_Z_DIFF_STAGE1_2_Fill2'] = abs(train_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2'] - 
#                                 train_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'])

# train_data['HEAD_NORMAL_Z_DIFF_STAGE2_3_Fill2'] = abs(train_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'] - 
#                                 train_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'])

# train_data['HEAD_NORMAL_Z_DIFF_STAGE1_3_Fill2'] = abs(train_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2'] - 
#                                 train_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'])

In [796]:
# test_data['HEAD_NORMAL_X_DIFF_STAGE1_2_Fill2'] = abs(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] - 
#                                 test_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'])

# test_data['HEAD_NORMAL_X_DIFF_STAGE2_3_Fill2'] = abs(test_data['HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2'] - 
#                                 test_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'])

# test_data['HEAD_NORMAL_X_DIFF_STAGE1_3_Fill2'] = abs(test_data['HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2'] - 
#                                 test_data['HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2'])


# test_data['HEAD_NORMAL_Y_DIFF_STAGE1_2_Fill2'] = abs(test_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2'] - 
#                                 train_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2'])

# test_data['HEAD_NORMAL_Y_DIFF_STAGE2_3_Fill2'] = abs(test_data['HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2'] - 
#                                 test_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2'])

# test_data['HEAD_NORMAL_Y_DIFF_STAGE1_3_Fill2'] = abs(test_data['HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2'] - 
#                                 test_data['HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2'])


# test_data['HEAD_NORMAL_Z_DIFF_STAGE1_2_Fill2'] = abs(test_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2'] - 
#                                 test_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'])

# test_data['HEAD_NORMAL_Z_DIFF_STAGE2_3_Fill2'] = abs(test_data['HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2'] - 
#                                 test_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'])

# test_data['HEAD_NORMAL_Z_DIFF_STAGE1_3_Fill2'] = abs(test_data['HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2'] - 
#                                 test_data['HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2'])


In [797]:
train_data['PRESURE_DIFF_1_2_AutoClave'] = abs(train_data['1st Pressure Collect Result_AutoClave'] - 
                                train_data['2nd Pressure Collect Result_AutoClave'])

train_data['PRESURE_DIFF_2_3_AutoClave'] = abs(train_data['2nd Pressure Collect Result_AutoClave'] - 
                                train_data['3rd Pressure Collect Result_AutoClave'])

train_data['PRESURE_DIFF_1_3_AutoClave'] = abs(train_data['1st Pressure Collect Result_AutoClave'] - 
                                train_data['3rd Pressure Collect Result_AutoClave'])

test_data['PRESURE_DIFF_1_2_AutoClave'] = abs(test_data['1st Pressure Collect Result_AutoClave'] - 
                                test_data['2nd Pressure Collect Result_AutoClave'])

test_data['PRESURE_DIFF_2_3_AutoClave'] = abs(test_data['2nd Pressure Collect Result_AutoClave'] - 
                                test_data['3rd Pressure Collect Result_AutoClave'])

test_data['PRESURE_DIFF_1_3_AutoClave'] = abs(test_data['1st Pressure Collect Result_AutoClave'] - 
                                test_data['3rd Pressure Collect Result_AutoClave'])



/tmp/ipykernel_149/3772767449.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['PRESURE_DIFF_1_2_AutoClave'] = abs(train_data['1st Pressure Collect Result_AutoClave'] -
/tmp/ipykernel_149/3772767449.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['PRESURE_DIFF_2_3_AutoClave'] = abs(train_data['2nd Pressure Collect Result_AutoClave'] -
/tmp/ipykernel_149/3772767449.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

In [798]:
missing_values = train_data.isnull().sum()

# 결측치가 있는 열만 선택
missing_columns = missing_values[missing_values >0]
missing_columns

Series([], dtype: int64)

In [799]:
missing_values = test_data.isnull().sum()

# 결측치가 있는 열만 선택
missing_columns = missing_values[missing_values >0]
missing_columns

Series([], dtype: int64)

In [800]:
columns_train = set(train_data.columns)
columns_test = set(test_data.columns)

# train에는 있고 test에는 없는 컬럼
diff_in_train = columns_train - columns_test
print(diff_in_train)
# test에는 있고 test에는 없는 컬럼
diff_in_test = columns_test - columns_train
print(diff_in_test)

{'target'}
{'Set ID'}


### 중복 데이터 열 삭제

In [801]:
seen_columns = {}
columns_to_drop = []

for col in train_data.columns:
    col_data = train_data[col].tolist()
    for seen_col, seen_data in seen_columns.items():
        if col_data == seen_data:
            columns_to_drop.append(col)
            break
    else:
        seen_columns[col] = col_data

# 중복된 열 삭제
train_data = train_data.drop(columns=columns_to_drop)
train_data

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,...,target,DISPENSE_VOLUME_DIFF_STAGE1_2_DAM,DISPENSE_VOLUME_DIFF_STAGE2_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1,DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1,DISPENSE_VOLUME_DIFF_STAGE1_3_Fill1,PRESURE_DIFF_1_2_AutoClave,PRESURE_DIFF_2_3_AutoClave,PRESURE_DIFF_1_3_AutoClave
0,Dam dispenser #1,AJX75334505,4F1XA938-1,240.0,2.5,-90,100,1030,16,14.9,...,Normal,0.46,0.44,0.02,8.64,8.55,0.09,0.181,0.006,0.187
1,Dam dispenser #1,AJX75334505,3KPM0016-2,240.0,2.5,-90,70,1030,10,21.3,...,Normal,1.15,1.15,0.00,9.40,9.40,0.00,0.000,0.187,0.187
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1000.0,12.5,90,85,280,16,14.7,...,Normal,0.68,0.68,0.00,9.50,9.41,0.09,0.175,0.008,0.183
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1000.0,12.5,90,70,280,10,21.3,...,Normal,0.91,0.91,0.00,9.40,9.40,0.00,0.001,0.200,0.201
4,Dam dispenser #1,AJX75334501,3HPM0007-1,240.0,2.5,-90,70,1030,10,9.7,...,Normal,0.33,0.33,0.00,9.40,9.40,0.00,0.000,0.196,0.196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,240.0,2.5,-90,70,1030,10,17.0,...,Normal,0.84,0.84,0.00,9.40,9.40,0.00,0.000,0.200,0.200
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1000.0,12.5,90,100,280,16,14.9,...,Normal,0.45,0.43,0.02,9.40,9.40,0.00,0.188,0.005,0.193
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,240.0,2.5,-90,100,1030,16,14.2,...,Normal,0.42,0.43,0.01,8.83,8.83,0.00,0.181,0.008,0.189
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1000.0,12.5,90,70,280,10,9.7,...,Normal,0.33,0.33,0.00,9.40,9.31,0.09,0.001,0.196,0.197


In [802]:
seen_columns = {}
columns_to_drop = []

for col in test_data.columns:
    col_data = test_data[col].tolist()
    for seen_col, seen_data in seen_columns.items():
        if col_data == seen_data:
            columns_to_drop.append(col)
            break
    else:
        seen_columns[col] = col_data

# 중복된 열 삭제
columns_to_drop.remove('Receip No Collect Result_Fill1')
test_data = test_data.drop(columns=columns_to_drop)
test_data

,Set ID,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,...,Production Qty Collect Result_Fill2,DISPENSE_VOLUME_DIFF_STAGE1_2_DAM,DISPENSE_VOLUME_DIFF_STAGE2_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1,DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1,DISPENSE_VOLUME_DIFF_STAGE1_3_Fill1,PRESURE_DIFF_1_2_AutoClave,PRESURE_DIFF_2_3_AutoClave,PRESURE_DIFF_1_3_AutoClave
0,0001be084fbc4aaa9d921f39e595961b,Dam dispenser #2,AJX75334501,3J1XF767-1,1000.0,12.5,90,70,280,10,...,195,0.85,0.85,0.0,9.40,9.40,0.0,0.001,0.182,0.183
1,0005bbd180064abd99e63f9ed3e1ac80,Dam dispenser #2,AJX75334501,4B1XD472-2,1000.0,12.5,90,70,280,16,...,256,0.41,0.41,0.0,8.83,8.83,0.0,0.279,0.017,0.296
2,000948934c4140d883d670adcb609584,Dam dispenser #1,AJX75334501,3H1XE355-1,240.0,2.5,-90,70,1030,10,...,98,0.33,0.33,0.0,9.31,9.21,0.1,0.000,0.192,0.192
3,000a6bfd02874c6296dc7b2e9c5678a7,Dam dispenser #2,AJX75334501,3L1XA128-1,1000.0,12.5,90,70,280,10,...,0,0.75,0.75,0.0,9.02,9.02,0.0,0.001,0.198,0.199
4,0018e78ce91343678716e2ea27a51c95,Dam dispenser #1,AJX75334501,4A1XA639-1,240.0,2.5,-90,70,1030,16,...,215,0.40,0.40,0.0,8.83,8.83,0.0,0.190,0.005,0.195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,ffea508b59934d689b540f95eb3fa730,Dam dispenser #2,AJX75334501,3K1XB597-1,1000.0,12.5,90,70,280,10,...,131,0.75,0.75,0.0,9.40,9.40,0.0,0.001,0.189,0.190
17357,ffed8923c8a448a98afc641b770be153,Dam dispenser #2,AJX75334501,4A1XB974-1,1000.0,12.5,90,70,280,16,...,279,0.39,0.39,0.0,8.83,8.83,0.0,0.190,0.005,0.195
17358,fff1e73734da40adbe805359b3efb462,Dam dispenser #1,AJX75334501,3L1XA998-1,240.0,2.5,-90,70,1030,16,...,66,0.73,0.73,0.0,9.03,8.93,0.1,0.000,0.189,0.189
17359,fff8e38bdd09470baf95f71e92075dec,Dam dispenser #1,AJX75334501,3F1XC376-1,240.0,2.5,-90,70,1030,10,...,117,0.40,0.40,0.0,7.79,7.79,0.0,0.001,0.194,0.195


In [803]:
columns_train = set(train_data.columns)
columns_test = set(test_data.columns)

# train에는 있고 test에는 없는 컬럼
diff_in_train = columns_train - columns_test
print(diff_in_train)
# test에는 있고 test에는 없는 컬럼
diff_in_test = columns_test - columns_train
print(diff_in_test)

{'target'}
{'Set ID'}


### 라벨 인코딩

In [804]:
label_columns =[]

for i in train_data.columns:
    if train_data[i].dtypes == 'object':
        label_columns.append(i)

In [805]:
label_columns

['Equipment_Dam',
 'Model.Suffix_Dam',
 'Workorder_Dam',
 'Chamber Temp. Judge Value_AutoClave',
 'Equipment_Fill1',
 'Equipment_Fill2',
 'target']

In [806]:
label_columns =[]

for i in test_data.columns:
    if test_data[i].dtypes == 'object':
        label_columns.append(i)

In [807]:
label_columns

['Set ID',
 'Equipment_Dam',
 'Model.Suffix_Dam',
 'Workorder_Dam',
 'Chamber Temp. Judge Value_AutoClave',
 'Equipment_Fill1',
 'Equipment_Fill2']

In [808]:
label_encoder = LabelEncoder()

In [809]:
combined_data = pd.concat([train_data, test_data], axis=0)

# 공통 object 형 열 라벨 인코딩
columns_to_encode = [
    'Equipment_Dam',
    'Model.Suffix_Dam',
    'Workorder_Dam',
    'Chamber Temp. Judge Value_AutoClave',
    'Equipment_Fill1',
    'Equipment_Fill2',
]

for col in columns_to_encode:
    combined_data[col] = label_encoder.fit_transform(combined_data[col])

In [810]:
train_data_encoded = combined_data.iloc[:len(train_data), :]
test_data_encoded = combined_data.iloc[len(train_data):, :]

In [811]:
train_data_encoded = train_data_encoded.drop(columns='Set ID')
train_data_encoded

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,...,target,DISPENSE_VOLUME_DIFF_STAGE1_2_DAM,DISPENSE_VOLUME_DIFF_STAGE2_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1,DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1,DISPENSE_VOLUME_DIFF_STAGE1_3_Fill1,PRESURE_DIFF_1_2_AutoClave,PRESURE_DIFF_2_3_AutoClave,PRESURE_DIFF_1_3_AutoClave
0,0,3,658,240.0,2.5,-90,100,1030,16,14.9,...,Normal,0.46,0.44,0.02,8.64,8.55,0.09,0.181,0.006,0.187
1,0,3,284,240.0,2.5,-90,70,1030,10,21.3,...,Normal,1.15,1.15,0.00,9.40,9.40,0.00,0.000,0.187,0.187
2,1,0,590,1000.0,12.5,90,85,280,16,14.7,...,Normal,0.68,0.68,0.00,9.50,9.41,0.09,0.175,0.008,0.183
3,1,0,252,1000.0,12.5,90,70,280,10,21.3,...,Normal,0.91,0.91,0.00,9.40,9.40,0.00,0.001,0.200,0.201
4,0,0,143,240.0,2.5,-90,70,1030,10,9.7,...,Normal,0.33,0.33,0.00,9.40,9.40,0.00,0.000,0.196,0.196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,0,0,239,240.0,2.5,-90,70,1030,10,17.0,...,Normal,0.84,0.84,0.00,9.40,9.40,0.00,0.000,0.200,0.200
40502,1,0,644,1000.0,12.5,90,100,280,16,14.9,...,Normal,0.45,0.43,0.02,9.40,9.40,0.00,0.188,0.005,0.193
40503,0,0,541,240.0,2.5,-90,100,1030,16,14.2,...,Normal,0.42,0.43,0.01,8.83,8.83,0.00,0.181,0.008,0.189
40504,1,0,165,1000.0,12.5,90,70,280,10,9.7,...,Normal,0.33,0.33,0.00,9.40,9.31,0.09,0.001,0.196,0.197


In [812]:
test_data_encoded = test_data_encoded.drop(columns='target')
test_data_encoded

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,...,DISPENSE_VOLUME_DIFF_STAGE1_2_DAM,DISPENSE_VOLUME_DIFF_STAGE2_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1,DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1,DISPENSE_VOLUME_DIFF_STAGE1_3_Fill1,PRESURE_DIFF_1_2_AutoClave,PRESURE_DIFF_2_3_AutoClave,PRESURE_DIFF_1_3_AutoClave,Set ID
0,1,0,241,1000.0,12.5,90,70,280,10,17.0,...,0.85,0.85,0.0,9.40,9.40,0.0,0.001,0.182,0.183,0001be084fbc4aaa9d921f39e595961b
1,1,0,510,1000.0,12.5,90,70,280,16,14.2,...,0.41,0.41,0.0,8.83,8.83,0.0,0.279,0.017,0.296,0005bbd180064abd99e63f9ed3e1ac80
2,0,0,128,240.0,2.5,-90,70,1030,10,9.7,...,0.33,0.33,0.0,9.31,9.21,0.1,0.000,0.192,0.192,000948934c4140d883d670adcb609584
3,1,0,307,1000.0,12.5,90,70,280,10,21.3,...,0.75,0.75,0.0,9.02,9.02,0.0,0.001,0.198,0.199,000a6bfd02874c6296dc7b2e9c5678a7
4,0,0,416,240.0,2.5,-90,70,1030,16,13.2,...,0.40,0.40,0.0,8.83,8.83,0.0,0.190,0.005,0.195,0018e78ce91343678716e2ea27a51c95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,1,0,278,1000.0,12.5,90,70,280,10,21.3,...,0.75,0.75,0.0,9.40,9.40,0.0,0.001,0.189,0.190,ffea508b59934d689b540f95eb3fa730
17357,1,0,440,1000.0,12.5,90,70,280,16,13.2,...,0.39,0.39,0.0,8.83,8.83,0.0,0.190,0.005,0.195,ffed8923c8a448a98afc641b770be153
17358,0,0,315,240.0,2.5,-90,70,1030,16,13.2,...,0.73,0.73,0.0,9.03,8.93,0.1,0.000,0.189,0.189,fff1e73734da40adbe805359b3efb462
17359,0,0,8,240.0,2.5,-90,70,1030,10,9.7,...,0.40,0.40,0.0,7.79,7.79,0.0,0.001,0.194,0.195,fff8e38bdd09470baf95f71e92075dec


## 3. 모델 학습


### 모델 정의

In [813]:
train_data_encoded

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,...,target,DISPENSE_VOLUME_DIFF_STAGE1_2_DAM,DISPENSE_VOLUME_DIFF_STAGE2_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1,DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1,DISPENSE_VOLUME_DIFF_STAGE1_3_Fill1,PRESURE_DIFF_1_2_AutoClave,PRESURE_DIFF_2_3_AutoClave,PRESURE_DIFF_1_3_AutoClave
0,0,3,658,240.0,2.5,-90,100,1030,16,14.9,...,Normal,0.46,0.44,0.02,8.64,8.55,0.09,0.181,0.006,0.187
1,0,3,284,240.0,2.5,-90,70,1030,10,21.3,...,Normal,1.15,1.15,0.00,9.40,9.40,0.00,0.000,0.187,0.187
2,1,0,590,1000.0,12.5,90,85,280,16,14.7,...,Normal,0.68,0.68,0.00,9.50,9.41,0.09,0.175,0.008,0.183
3,1,0,252,1000.0,12.5,90,70,280,10,21.3,...,Normal,0.91,0.91,0.00,9.40,9.40,0.00,0.001,0.200,0.201
4,0,0,143,240.0,2.5,-90,70,1030,10,9.7,...,Normal,0.33,0.33,0.00,9.40,9.40,0.00,0.000,0.196,0.196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,0,0,239,240.0,2.5,-90,70,1030,10,17.0,...,Normal,0.84,0.84,0.00,9.40,9.40,0.00,0.000,0.200,0.200
40502,1,0,644,1000.0,12.5,90,100,280,16,14.9,...,Normal,0.45,0.43,0.02,9.40,9.40,0.00,0.188,0.005,0.193
40503,0,0,541,240.0,2.5,-90,100,1030,16,14.2,...,Normal,0.42,0.43,0.01,8.83,8.83,0.00,0.181,0.008,0.189
40504,1,0,165,1000.0,12.5,90,70,280,10,9.7,...,Normal,0.33,0.33,0.00,9.40,9.31,0.09,0.001,0.196,0.197


In [814]:
# # 'Workorder_Dam' 열을 기준으로 그룹화하고, 각 그룹에 대해 'target' 열의 값이 모두 'Normal'인지 확인
# train_data_encoded['All_Normal'] = train_data_encoded.groupby('Workorder_Dam')['target'] \
#                                                       .transform(lambda x: 0 if (x == 'Normal').all() else 1)

# # 결과 확인
# train_data_encoded['All_Normal'].value_counts()

In [815]:
# workorder_dam_normal_status = train_data_encoded.groupby('Workorder_Dam')['target'] \
#                                                 .apply(lambda x: 0 if (x == 'Normal').all() else 1) \
#                                                 .reset_index()

# # 열 이름 변경
# workorder_dam_normal_status.columns = ['Workorder_Dam', 'All_Normal']
# workorder_dam_normal_status['All_Normal'].value_counts()

In [816]:
# test_data_encoded = test_data_encoded.merge(workorder_dam_normal_status, on='Workorder_Dam', how='left')

# # All_Normal 값이 NaN인 경우 0으로 채우기 (train 데이터에 없는 Workorder_Dam에 대한 기본 처리)
# test_data_encoded['All_Normal'].fillna(0, inplace=True)

# test_data_encoded['All_Normal'] = test_data_encoded['All_Normal'].apply(lambda x: 1 if x == 1.0 else (0 if x == 0.0 else x))

# # 결과 확인
# test_data_encoded['All_Normal'].value_counts()

In [817]:
features = []

for i in train_data_encoded.columns:
    features.append(i)
    
features.remove('target')

In [818]:
train_x =train_data_encoded[features]
train_y = train_data_encoded["target"]

In [819]:
from imblearn.under_sampling import TomekLinks

smote = TomekLinks()
train_x_smote, train_y_smote = smote.fit_resample(train_x, train_y)
print("SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ", train_x.shape, train_y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :', train_x_smote.shape, train_y_smote.shape)
print('SMOTE 적용 전 값의 분포 :\n',pd.Series(train_y).value_counts() )
print('SMOTE 적용 후 값의 분포 :\n',pd.Series(train_y_smote).value_counts() )

SMOTE 적용 전 학습용 피처/레이블 데이터 세트 :  (40505, 115) (40505,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트 : (39406, 115) (39406,)
SMOTE 적용 전 값의 분포 :
 target
Normal      38156
AbNormal     2349
Name: count, dtype: int64
SMOTE 적용 후 값의 분포 :
 target
Normal      37057
AbNormal     2349
Name: count, dtype: int64


In [820]:
train_x = train_x_smote
train_y = train_y_smote

In [821]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39406 entries, 0 to 39405
Columns: 115 entries, Equipment_Dam to PRESURE_DIFF_1_3_AutoClave
dtypes: float64(76), int64(39)
memory usage: 34.6 MB


In [822]:
train_x['CURE END POSITION X Collect Result_Dam'] = train_x['CURE END POSITION X Collect Result_Dam'].astype(int)
train_x['PalletID Collect Result_Dam'] = train_x['PalletID Collect Result_Dam'].astype(int)
train_x['PalletID Collect Result_Fill1'] = train_x['PalletID Collect Result_Fill1'].astype(int)
train_x['PalletID Collect Result_Fill2'] = train_x['PalletID Collect Result_Fill2'].astype(int)

In [823]:
int_columns_indice = [index for index, dtype in enumerate(train_x.dtypes) if dtype == 'int64']
#[0,1,2,5,6,7,8,9,119] Equipment_Fill1 Equipment_Fill2 Chamber Temp. Judge Value_AutoClave

In [824]:
model = CatBoostClassifier(verbose=0, cat_features =int_columns_indice,random_state=RANDOM_STATE, scale_pos_weight = 0.10)

### 모델 학습


In [825]:
model.fit(train_x, train_y)

### feature importance 확인 후 0인 열 제거

In [826]:
# 123개 중 int 다 포함 하고 0.1 cat모델을 fit 하고 난뒤의 importace 그리고 여기서 0인것들 삭제해서 다시함
feature_importances = model.get_feature_importance(Pool(train_x, label=train_y, cat_features=int_columns_indice))

In [827]:
feature_importance_df = pd.DataFrame({'Feature': train_x.columns, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [828]:
feature_importance_df.head(60)

,Feature,Importance
2,Workorder_Dam,9.152594
56,Chamber Temp. Collect Result_AutoClave,7.006727
34,Production Qty Collect Result_Dam,5.341284
105,Production Qty Collect Result_Fill2,4.765876
82,Production Qty Collect Result_Fill1,4.185375
103,Machine Tact time Collect Result_Fill2,3.731201
32,Machine Tact time Collect Result_Dam,3.685707
33,PalletID Collect Result_Dam,3.603076
80,Machine Tact time Collect Result_Fill1,3.415478
114,PRESURE_DIFF_1_3_AutoClave,3.309428


In [829]:
zero_importance_features = feature_importance_df[feature_importance_df['Importance'] == 0]['Feature'].tolist()

In [830]:
zero_importance_features

['Head Clean Position X Collect Result_Dam',
 'HEAD Standby Position Z Collect Result_Dam',
 'HEAD Standby Position X Collect Result_Fill1',
 'WorkMode Collect Result_Dam',
 'Head Zero Position X Collect Result_Dam',
 'CURE END POSITION Θ Collect Result_Dam',
 'CURE END POSITION Z Collect Result_Dam',
 'CURE END POSITION X Collect Result_Dam',
 'HEAD Standby Position Y Collect Result_Dam',
 'HEAD Standby Position X Collect Result_Dam',
 'Head Clean Position Z Collect Result_Fill2',
 'Head Clean Position X Collect Result_Fill2',
 'HEAD Standby Position Z Collect Result_Fill2',
 'HEAD Standby Position X Collect Result_Fill2',
 'HEAD Standby Position Y Collect Result_Fill1',
 'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2',
 'CURE START POSITION X Collect Result_Fill2',
 'Equipment_Fill1',
 'Equipment_Fill2',
 'Head Clean Position Z Collect Result_Fill1',
 'Head Clean Position X Collect Result_Fill1',
 'Equipment_Dam']

In [831]:
zero_importance_features.remove('Equipment_Fill1')
#zero_importance_features.remove('Equipment_Fill2')
#zero_importance_features.remove('Equipment_Dam')

In [834]:
# def objective(trial):
#     params = {
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
#         'iterations': trial.suggest_int('iterations', 300, 2000),
#         'depth': trial.suggest_int('depth', 4, 12),
#         'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.09,0.13, log=True),
#     }
    
#     model = CatBoostClassifier(verbose=0, cat_features =int_columns_indices,random_state=RANDOM_STATE)   
    
#     f1_macro_scores = []

#     for train_idx, valid_idx in skf.split(train_x, train_y):
#         train_data = train_x.iloc[train_idx]
#         train_labels = train_y.iloc[train_idx]

#         valid_data = train_x.iloc[valid_idx]
#         valid_labels = train_y.iloc[valid_idx]

#         model.fit(train_data,train_labels)

#         predictions = model.predict(valid_data)

#         f1_macro = f1_score(valid_labels, predictions, average='micro')
#         f1_macro_scores.append(f1_macro)
    

#     # 각 fold의 평균 f1-score를 반환합니다.
#     return np.mean(f1_macro_scores)

# # Optuna Study 객체 생성
# study = optuna.create_study(direction='maximize')

# # 하이퍼파라미터 최적화 실행
# study.optimize(objective, n_trials=100)

# # 최적화된 결과 출력
# print('Best trial:')
# best_trial = study.best_trial
# print('  Value: {}'.format(best_trial.value))
# print('  Params: ')
# for key, value in best_trial.params.items():
#     print('    {}: {}'.format(key, value))
# # 'learning_rate': 0.13269403227093554, 'iterations': 1349, 'depth': 8, 'scale_pos_weight': 0.10665962498304488
# # 이상함 다 똑같이 뜸

In [833]:
train_x = train_x.drop(zero_importance_features, axis=1)
train_x

,Model.Suffix_Dam,Workorder_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,...,Production Qty Collect Result_Fill2,DISPENSE_VOLUME_DIFF_STAGE1_2_DAM,DISPENSE_VOLUME_DIFF_STAGE2_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1,DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1,DISPENSE_VOLUME_DIFF_STAGE1_3_Fill1,PRESURE_DIFF_1_2_AutoClave,PRESURE_DIFF_2_3_AutoClave,PRESURE_DIFF_1_3_AutoClave
0,3,284,70,1030,10,21.3,4.9,21.3,1.49,0.34,...,185,1.15,1.15,0.00,9.40,9.40,0.00,0.000,0.187,0.187
1,0,590,85,280,16,14.7,8.5,14.7,1.61,0.93,...,73,0.68,0.68,0.00,9.50,9.41,0.09,0.175,0.008,0.183
2,0,252,70,280,10,21.3,8.4,21.3,1.49,0.58,...,268,0.91,0.91,0.00,9.40,9.40,0.00,0.001,0.200,0.201
3,0,143,70,1030,10,9.7,4.9,9.6,0.67,0.34,...,121,0.33,0.33,0.00,9.40,9.40,0.00,0.000,0.196,0.196
4,0,24,70,1030,10,9.6,4.9,9.7,0.67,0.34,...,343,0.33,0.33,0.00,8.07,8.07,0.00,0.185,0.000,0.185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39401,0,239,70,1030,10,17.0,5.0,17.0,1.19,0.35,...,318,0.84,0.84,0.00,9.40,9.40,0.00,0.000,0.200,0.200
39402,0,644,100,280,16,14.9,8.5,14.7,1.04,0.59,...,197,0.45,0.43,0.02,9.40,9.40,0.00,0.188,0.005,0.193
39403,0,541,100,1030,16,14.2,8.2,14.3,0.99,0.57,...,27,0.42,0.43,0.01,8.83,8.83,0.00,0.181,0.008,0.189
39404,0,165,70,280,10,9.7,4.9,9.7,0.67,0.34,...,117,0.33,0.33,0.00,9.40,9.31,0.09,0.001,0.196,0.197


In [835]:
# train_x_0 = train_data_encoded[train_data_encoded['Equipment_Dam'] == 0][features]
# train_x_0 = train_x_0.drop(zero_importance_features, axis=1)
# train_y_0 = train_data_encoded[train_data_encoded['Equipment_Dam'] == 0]["target"]

# train_x_1 = train_data_encoded[train_data_encoded['Equipment_Dam'] == 1][features]
# train_x_1 = train_x_1.drop(zero_importance_features, axis=1)
# train_y_1 = train_data_encoded[train_data_encoded['Equipment_Dam'] == 1]["target"]

### 모델 재정의

In [836]:
int_columns_indices = [index for index, dtype in enumerate(train_x.dtypes) if dtype == 'int64']

In [837]:
for index in int_columns_indices:
    column_name = train_x.columns[index]
    train_x[column_name] = train_x[column_name].astype('category')

In [838]:
int_columns_indices = [index for index, dtype in enumerate(train_x.dtypes) if dtype == 'int64' or dtype=='category']

In [839]:
model = CatBoostClassifier(cat_features =int_columns_indices, random_state=RANDOM_STATE, scale_pos_weight = 0.11,
#                           iterations=1050,
                          ) 

In [840]:
# estimators = [
#     ('cat', model)
# ]
# clf = StackingClassifier(
#     estimators=estimators, final_estimator=model, cv=skf
# )

In [841]:
# clf.fit(train_x, train_y)

In [842]:
model_xgb = XGBClassifier(enable_categorical=True, random_state=RANDOM_STATE, scale_pos_weight=0.08)

In [843]:
# class_weights = {'Normal': 1, "AbNormal": 10.5}
model_lgbm = LGBMClassifier(verbose=0, random_state=RANDOM_STATE, scale_pos_weight=0.07)

In [844]:
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=9,  metric='cosine')

In [845]:
voting = VotingClassifier(
    estimators=[
         ('cat',model),
#        ('xgb', model_xgb),
#         ('lgbm', model_lgbm),
#         ('dt', model_dt),
        ('knn', model_knn)
    ],
#     weights = [2,1,1],
#     final_estimator = model_lgbm
    voting='hard',
    
)


In [846]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

In [847]:
f1_macro_scores = []

for train_idx, valid_idx in skf.split(train_x, train_y):
    train_data = train_x.iloc[train_idx]
    train_labels = train_y.iloc[train_idx]

    valid_data = train_x.iloc[valid_idx]
    valid_labels = train_y.iloc[valid_idx]

    voting.fit(train_data,train_labels) # eval_set=[(valid_data, valid_labels )], early_stopping_rounds=100)

    predictions = voting.predict(valid_data)

    f1_macro = f1_score(valid_labels, predictions, average='micro')
    f1_macro_scores.append(f1_macro)

average_f1_macro = np.mean(f1_macro_scores)

# fold 별 f1 score 와 평균 f1 score 출력을 모델 안정성 확인
print("Average F1-macro score:", average_f1_macro)
print(f1_macro_scores)

Learning rate set to 0.044963
0:	learn: 0.6877743	total: 141ms	remaining: 2m 20s
1:	learn: 0.6829430	total: 247ms	remaining: 2m 3s
2:	learn: 0.6768905	total: 385ms	remaining: 2m 7s
3:	learn: 0.6723823	total: 508ms	remaining: 2m 6s
4:	learn: 0.6672119	total: 626ms	remaining: 2m 4s
5:	learn: 0.6636479	total: 754ms	remaining: 2m 4s
6:	learn: 0.6595193	total: 890ms	remaining: 2m 6s
7:	learn: 0.6553906	total: 1.02s	remaining: 2m 6s
8:	learn: 0.6517521	total: 1.14s	remaining: 2m 5s
9:	learn: 0.6481159	total: 1.27s	remaining: 2m 5s
10:	learn: 0.6449590	total: 1.37s	remaining: 2m 3s
11:	learn: 0.6420162	total: 1.5s	remaining: 2m 3s
12:	learn: 0.6398091	total: 1.57s	remaining: 1m 59s
13:	learn: 0.6366243	total: 1.67s	remaining: 1m 57s
14:	learn: 0.6344242	total: 1.78s	remaining: 1m 57s
15:	learn: 0.6321411	total: 1.91s	remaining: 1m 57s
16:	learn: 0.6296485	total: 2.02s	remaining: 1m 56s
17:	learn: 0.6276889	total: 2.16s	remaining: 1m 57s
18:	learn: 0.6258786	total: 2.26s	remaining: 1m 56s
19:	

158:	learn: 0.5612083	total: 14.2s	remaining: 1m 15s
159:	learn: 0.5609600	total: 14.3s	remaining: 1m 15s
160:	learn: 0.5607825	total: 14.4s	remaining: 1m 15s
161:	learn: 0.5605290	total: 14.5s	remaining: 1m 14s
162:	learn: 0.5603790	total: 14.6s	remaining: 1m 14s
163:	learn: 0.5602167	total: 14.6s	remaining: 1m 14s
164:	learn: 0.5600558	total: 14.7s	remaining: 1m 14s
165:	learn: 0.5597900	total: 14.8s	remaining: 1m 14s
166:	learn: 0.5594704	total: 14.9s	remaining: 1m 14s
167:	learn: 0.5590112	total: 15s	remaining: 1m 14s
168:	learn: 0.5587078	total: 15.1s	remaining: 1m 14s
169:	learn: 0.5583067	total: 15.2s	remaining: 1m 14s
170:	learn: 0.5580547	total: 15.3s	remaining: 1m 13s
171:	learn: 0.5579767	total: 15.4s	remaining: 1m 13s
172:	learn: 0.5577614	total: 15.4s	remaining: 1m 13s
173:	learn: 0.5576210	total: 15.5s	remaining: 1m 13s
174:	learn: 0.5572132	total: 15.6s	remaining: 1m 13s
175:	learn: 0.5571429	total: 15.7s	remaining: 1m 13s
176:	learn: 0.5569870	total: 15.8s	remaining: 1m

317:	learn: 0.5186576	total: 27.6s	remaining: 59.1s
318:	learn: 0.5181550	total: 27.6s	remaining: 59s
319:	learn: 0.5177951	total: 27.7s	remaining: 58.9s
320:	learn: 0.5174348	total: 27.8s	remaining: 58.8s
321:	learn: 0.5169882	total: 27.9s	remaining: 58.7s
322:	learn: 0.5167049	total: 28s	remaining: 58.6s
323:	learn: 0.5166193	total: 28.1s	remaining: 58.5s
324:	learn: 0.5164423	total: 28.1s	remaining: 58.5s
325:	learn: 0.5163555	total: 28.2s	remaining: 58.4s
326:	learn: 0.5158922	total: 28.3s	remaining: 58.3s
327:	learn: 0.5156482	total: 28.4s	remaining: 58.1s
328:	learn: 0.5152989	total: 28.5s	remaining: 58.1s
329:	learn: 0.5151068	total: 28.6s	remaining: 58s
330:	learn: 0.5145715	total: 28.6s	remaining: 57.9s
331:	learn: 0.5143864	total: 28.7s	remaining: 57.8s
332:	learn: 0.5140274	total: 28.8s	remaining: 57.7s
333:	learn: 0.5136330	total: 28.9s	remaining: 57.6s
334:	learn: 0.5133436	total: 29s	remaining: 57.5s
335:	learn: 0.5131879	total: 29.1s	remaining: 57.4s
336:	learn: 0.513105

476:	learn: 0.4780051	total: 40.7s	remaining: 44.6s
477:	learn: 0.4776709	total: 40.8s	remaining: 44.5s
478:	learn: 0.4773912	total: 40.9s	remaining: 44.4s
479:	learn: 0.4772672	total: 40.9s	remaining: 44.4s
480:	learn: 0.4770393	total: 41s	remaining: 44.3s
481:	learn: 0.4768369	total: 41.1s	remaining: 44.2s
482:	learn: 0.4766488	total: 41.2s	remaining: 44.1s
483:	learn: 0.4765016	total: 41.3s	remaining: 44s
484:	learn: 0.4763383	total: 41.3s	remaining: 43.9s
485:	learn: 0.4761068	total: 41.4s	remaining: 43.8s
486:	learn: 0.4758849	total: 41.5s	remaining: 43.7s
487:	learn: 0.4757489	total: 41.6s	remaining: 43.6s
488:	learn: 0.4754894	total: 41.6s	remaining: 43.5s
489:	learn: 0.4752646	total: 41.7s	remaining: 43.4s
490:	learn: 0.4750319	total: 41.8s	remaining: 43.3s
491:	learn: 0.4748087	total: 41.9s	remaining: 43.3s
492:	learn: 0.4746438	total: 42s	remaining: 43.2s
493:	learn: 0.4744080	total: 42.1s	remaining: 43.1s
494:	learn: 0.4742773	total: 42.2s	remaining: 43s
495:	learn: 0.474056

635:	learn: 0.4462260	total: 53.9s	remaining: 30.9s
636:	learn: 0.4459503	total: 54s	remaining: 30.8s
637:	learn: 0.4457434	total: 54.1s	remaining: 30.7s
638:	learn: 0.4455431	total: 54.2s	remaining: 30.6s
639:	learn: 0.4452095	total: 54.3s	remaining: 30.5s
640:	learn: 0.4450055	total: 54.3s	remaining: 30.4s
641:	learn: 0.4447371	total: 54.4s	remaining: 30.3s
642:	learn: 0.4444626	total: 54.5s	remaining: 30.3s
643:	learn: 0.4443544	total: 54.6s	remaining: 30.2s
644:	learn: 0.4442632	total: 54.7s	remaining: 30.1s
645:	learn: 0.4440172	total: 54.8s	remaining: 30s
646:	learn: 0.4438339	total: 54.9s	remaining: 29.9s
647:	learn: 0.4436824	total: 54.9s	remaining: 29.8s
648:	learn: 0.4435504	total: 55s	remaining: 29.8s
649:	learn: 0.4434784	total: 55.1s	remaining: 29.7s
650:	learn: 0.4433473	total: 55.2s	remaining: 29.6s
651:	learn: 0.4432020	total: 55.3s	remaining: 29.5s
652:	learn: 0.4429281	total: 55.4s	remaining: 29.4s
653:	learn: 0.4428161	total: 55.4s	remaining: 29.3s
654:	learn: 0.4426

...

297:	learn: 0.5188411	total: 23.5s	remaining: 55.4s
298:	learn: 0.5186054	total: 23.6s	remaining: 55.3s
299:	learn: 0.5184496	total: 23.7s	remaining: 55.2s
300:	learn: 0.5182691	total: 23.7s	remaining: 55.1s
301:	learn: 0.5180618	total: 23.8s	remaining: 55.1s
302:	learn: 0.5177375	total: 23.9s	remaining: 55s
303:	learn: 0.5172063	total: 24s	remaining: 54.9s
304:	learn: 0.5167895	total: 24.1s	remaining: 54.8s
305:	learn: 0.5164666	total: 24.1s	remaining: 54.7s
306:	learn: 0.5162017	total: 24.2s	remaining: 54.7s
307:	learn: 0.5158713	total: 24.3s	remaining: 54.6s
308:	learn: 0.5156432	total: 24.4s	remaining: 54.5s
309:	learn: 0.5153983	total: 24.5s	remaining: 54.4s
310:	learn: 0.5151331	total: 24.5s	remaining: 54.4s
311:	learn: 0.5145015	total: 24.6s	remaining: 54.3s
312:	learn: 0.5143147	total: 24.7s	remaining: 54.2s
313:	learn: 0.5138761	total: 24.8s	remaining: 54.2s
314:	learn: 0.5136704	total: 24.9s	remaining: 54.1s
315:	learn: 0.5135066	total: 24.9s	remaining: 54s
316:	learn: 0.5132

456:	learn: 0.4762646	total: 36.4s	remaining: 43.2s
457:	learn: 0.4759597	total: 36.5s	remaining: 43.1s
458:	learn: 0.4758349	total: 36.5s	remaining: 43s
459:	learn: 0.4756521	total: 36.6s	remaining: 43s
460:	learn: 0.4753627	total: 36.7s	remaining: 42.9s
461:	learn: 0.4750736	total: 36.8s	remaining: 42.8s
462:	learn: 0.4746473	total: 36.8s	remaining: 42.7s
463:	learn: 0.4743037	total: 36.9s	remaining: 42.7s
464:	learn: 0.4740111	total: 37s	remaining: 42.6s
465:	learn: 0.4737579	total: 37.1s	remaining: 42.5s
466:	learn: 0.4735086	total: 37.2s	remaining: 42.4s
467:	learn: 0.4733215	total: 37.3s	remaining: 42.3s
468:	learn: 0.4730996	total: 37.3s	remaining: 42.3s
469:	learn: 0.4729381	total: 37.4s	remaining: 42.2s
470:	learn: 0.4728250	total: 37.5s	remaining: 42.1s
471:	learn: 0.4726244	total: 37.6s	remaining: 42.1s
472:	learn: 0.4724427	total: 37.7s	remaining: 42s
473:	learn: 0.4723187	total: 37.8s	remaining: 41.9s
474:	learn: 0.4722291	total: 37.8s	remaining: 41.8s
475:	learn: 0.471890

615:	learn: 0.4418800	total: 49.4s	remaining: 30.8s
616:	learn: 0.4416660	total: 49.5s	remaining: 30.7s
617:	learn: 0.4415237	total: 49.6s	remaining: 30.6s
618:	learn: 0.4411814	total: 49.6s	remaining: 30.6s
619:	learn: 0.4408866	total: 49.7s	remaining: 30.5s
620:	learn: 0.4407103	total: 49.8s	remaining: 30.4s
621:	learn: 0.4405503	total: 49.9s	remaining: 30.3s
622:	learn: 0.4404298	total: 50s	remaining: 30.3s
623:	learn: 0.4402894	total: 50.1s	remaining: 30.2s
624:	learn: 0.4401744	total: 50.2s	remaining: 30.1s
625:	learn: 0.4398010	total: 50.3s	remaining: 30s
626:	learn: 0.4395369	total: 50.3s	remaining: 29.9s
627:	learn: 0.4393141	total: 50.4s	remaining: 29.9s
628:	learn: 0.4390688	total: 50.5s	remaining: 29.8s
629:	learn: 0.4387447	total: 50.6s	remaining: 29.7s
630:	learn: 0.4385467	total: 50.7s	remaining: 29.6s
631:	learn: 0.4382277	total: 50.8s	remaining: 29.6s
632:	learn: 0.4379534	total: 50.8s	remaining: 29.5s
633:	learn: 0.4379102	total: 50.9s	remaining: 29.4s
634:	learn: 0.43

777:	learn: 0.4104593	total: 1m 2s	remaining: 17.9s
778:	learn: 0.4102688	total: 1m 2s	remaining: 17.8s
779:	learn: 0.4100534	total: 1m 2s	remaining: 17.8s
780:	learn: 0.4098313	total: 1m 3s	remaining: 17.7s
781:	learn: 0.4095844	total: 1m 3s	remaining: 17.6s
782:	learn: 0.4095127	total: 1m 3s	remaining: 17.5s
783:	learn: 0.4092868	total: 1m 3s	remaining: 17.4s
784:	learn: 0.4089330	total: 1m 3s	remaining: 17.3s
785:	learn: 0.4087795	total: 1m 3s	remaining: 17.3s
786:	learn: 0.4086275	total: 1m 3s	remaining: 17.2s
787:	learn: 0.4082145	total: 1m 3s	remaining: 17.1s
788:	learn: 0.4079135	total: 1m 3s	remaining: 17s
789:	learn: 0.4077851	total: 1m 3s	remaining: 16.9s
790:	learn: 0.4076250	total: 1m 3s	remaining: 16.9s
791:	learn: 0.4075489	total: 1m 3s	remaining: 16.8s
792:	learn: 0.4073829	total: 1m 3s	remaining: 16.7s
793:	learn: 0.4070761	total: 1m 4s	remaining: 16.6s
794:	learn: 0.4067893	total: 1m 4s	remaining: 16.5s
795:	learn: 0.4065263	total: 1m 4s	remaining: 16.5s
796:	learn: 0.

936:	learn: 0.3836810	total: 1m 15s	remaining: 5.09s
937:	learn: 0.3834655	total: 1m 15s	remaining: 5.01s
938:	learn: 0.3833068	total: 1m 15s	remaining: 4.93s
939:	learn: 0.3831985	total: 1m 15s	remaining: 4.85s
940:	learn: 0.3831161	total: 1m 16s	remaining: 4.77s
941:	learn: 0.3830088	total: 1m 16s	remaining: 4.69s
942:	learn: 0.3829032	total: 1m 16s	remaining: 4.61s
943:	learn: 0.3826983	total: 1m 16s	remaining: 4.53s
944:	learn: 0.3824373	total: 1m 16s	remaining: 4.44s
945:	learn: 0.3823546	total: 1m 16s	remaining: 4.36s
946:	learn: 0.3820896	total: 1m 16s	remaining: 4.28s
947:	learn: 0.3819373	total: 1m 16s	remaining: 4.2s
948:	learn: 0.3818642	total: 1m 16s	remaining: 4.12s
949:	learn: 0.3816867	total: 1m 16s	remaining: 4.04s
950:	learn: 0.3816114	total: 1m 16s	remaining: 3.96s
951:	learn: 0.3812664	total: 1m 16s	remaining: 3.88s
952:	learn: 0.3811724	total: 1m 17s	remaining: 3.8s
953:	learn: 0.3809295	total: 1m 17s	remaining: 3.72s
954:	learn: 0.3806876	total: 1m 17s	remaining: 3

### 테스트 데이터 예측


테스트 데이터 불러오기

In [848]:
df_test_x = test_data_encoded[features]
df_test_x = df_test_x.drop(zero_importance_features, axis=1)
df_test_x

,Model.Suffix_Dam,Workorder_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,...,Production Qty Collect Result_Fill2,DISPENSE_VOLUME_DIFF_STAGE1_2_DAM,DISPENSE_VOLUME_DIFF_STAGE2_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1,DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1,DISPENSE_VOLUME_DIFF_STAGE1_3_Fill1,PRESURE_DIFF_1_2_AutoClave,PRESURE_DIFF_2_3_AutoClave,PRESURE_DIFF_1_3_AutoClave
0,0,241,70,280,10,17.0,4.9,17.0,1.19,0.34,...,195,0.85,0.85,0.0,9.40,9.40,0.0,0.001,0.182,0.183
1,0,510,70,280,16,14.2,8.3,14.2,0.99,0.58,...,256,0.41,0.41,0.0,8.83,8.83,0.0,0.279,0.017,0.296
2,0,128,70,1030,10,9.7,4.9,9.7,0.67,0.34,...,98,0.33,0.33,0.0,9.31,9.21,0.1,0.000,0.192,0.192
3,0,307,70,280,10,21.3,10.6,21.3,1.49,0.74,...,0,0.75,0.75,0.0,9.02,9.02,0.0,0.001,0.198,0.199
4,0,416,70,1030,16,13.2,7.5,13.2,0.92,0.52,...,215,0.40,0.40,0.0,8.83,8.83,0.0,0.190,0.005,0.195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,0,278,70,280,10,21.3,10.6,21.3,1.49,0.74,...,131,0.75,0.75,0.0,9.40,9.40,0.0,0.001,0.189,0.190
17357,0,440,70,280,16,13.2,7.6,13.2,0.92,0.53,...,279,0.39,0.39,0.0,8.83,8.83,0.0,0.190,0.005,0.195
17358,0,315,70,1030,16,13.2,6.6,13.2,1.45,0.72,...,66,0.73,0.73,0.0,9.03,8.93,0.1,0.000,0.189,0.189
17359,0,8,70,1030,10,9.7,3.9,9.7,0.67,0.27,...,117,0.40,0.40,0.0,7.79,7.79,0.0,0.001,0.194,0.195


In [849]:
# df_test_x['CURE END POSITION X Collect Result_Dam'] = df_test_x['CURE END POSITION X Collect Result_Dam'].astype(int)
df_test_x['PalletID Collect Result_Dam'] = df_test_x['PalletID Collect Result_Dam'].astype(int)
df_test_x['PalletID Collect Result_Fill1'] = df_test_x['PalletID Collect Result_Fill1'].astype(int)
df_test_x['PalletID Collect Result_Fill2'] = df_test_x['PalletID Collect Result_Fill2'].astype(int)

In [850]:
for index in int_columns_indices:
    column_name = df_test_x.columns[index]
    df_test_x[column_name] = df_test_x[column_name].astype('category')

In [851]:
# df_test_x['final_target'] = df_test_x.apply(lambda row: 'AbNormal' if 'AbNormal' in [row['target_0'], row['target_1']] else 'Normal', axis=1)

In [852]:
test_pred = voting.predict(df_test_x)

In [853]:
# test_pred_proba = model.predict_proba(df_test_x)
# test_pred_proba

In [854]:
df_test_x['target'] = test_pred

In [855]:
# test_pred[679] = 'AbNormal'
# test_pred[1739] = 'AbNormal'

In [856]:
# 0 1 60 은 Abnormal
df_test_x[(df_test_x['1st Pressure 1st Pressure Unit Time_AutoClave']==0)][['1st Pressure 1st Pressure Unit Time_AutoClave', '2nd Pressure Unit Time_AutoClave', '3rd Pressure Unit Time_AutoClave','target']]

,1st Pressure 1st Pressure Unit Time_AutoClave,2nd Pressure Unit Time_AutoClave,3rd Pressure Unit Time_AutoClave,target
478,0,181,120,Normal
679,0,1,60,AbNormal
1120,0,181,120,Normal
1739,0,1,60,AbNormal
2068,0,181,120,Normal
2292,0,181,120,Normal
2576,0,181,120,Normal
3674,0,181,120,Normal
6009,0,181,120,Normal
7056,0,181,120,Normal


In [857]:
# df_test_x.loc[df_test_x.index == 679, 'target' ] ='AbNormal'

In [858]:
df_test_x[df_test_x['Machine Tact time Collect Result_Fill1']<45] # 공정 시간이 너무 짧음.. 근데 46 초는 normal

,Model.Suffix_Dam,Workorder_Dam,CURE SPEED Collect Result_Dam,CURE START POSITION X Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage3) Collect Result_Dam,Dispense Volume(Stage1) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,...,DISPENSE_VOLUME_DIFF_STAGE1_2_DAM,DISPENSE_VOLUME_DIFF_STAGE2_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_3_DAM,DISPENSE_VOLUME_DIFF_STAGE1_2_Fill1,DISPENSE_VOLUME_DIFF_STAGE2_3_Fill1,DISPENSE_VOLUME_DIFF_STAGE1_3_Fill1,PRESURE_DIFF_1_2_AutoClave,PRESURE_DIFF_2_3_AutoClave,PRESURE_DIFF_1_3_AutoClave,target
2995,0,361,70,280,16,13.2,6.6,13.2,1.45,0.72,...,0.73,0.73,0.0,8.27,8.27,0.0,0.002,0.200,0.202,Normal
5402,0,538,95,1030,16,14.7,8.4,14.7,1.02,0.58,...,0.44,0.44,0.0,9.31,9.31,0.0,0.177,0.008,0.185,Normal
8398,0,343,70,1030,16,13.2,6.5,13.2,1.45,0.71,...,0.74,0.74,0.0,8.65,8.65,0.0,0.000,0.188,0.188,Normal
11725,0,90,70,1030,10,9.7,4.9,9.7,0.67,0.34,...,0.33,0.33,0.0,9.40,9.40,0.0,0.001,0.190,0.191,Normal
13592,0,349,70,280,16,13.2,6.6,13.2,1.45,0.72,...,0.73,0.73,0.0,8.36,8.46,0.1,0.000,0.191,0.191,Normal


In [859]:
# df_test_x[(df_test_x['3rd Pressure Collect Result_AutoClave'] >0.0) & (df_test_x['Chamber Temp. Judge Value_AutoClave']==0)][['1st Pressure Collect Result_AutoClave',
#                 '1st Pressure 1st Pressure Unit Time_AutoClave','2nd Pressure Collect Result_AutoClave',
#                 '2nd Pressure Unit Time_AutoClave','3rd Pressure Collect Result_AutoClave',
#                 '3rd Pressure Unit Time_AutoClave', 'Chamber Temp. Collect Result_AutoClave',
#                 'Chamber Temp. Unit Time_AutoClave','Chamber Temp. Judge Value_AutoClave', 'Model.Suffix_Dam', 'Workorder_Dam', 'target']]
# smotetomek cat fit 1956

# 8/21
# selectfromModel 패키지(cat 0.1 로판단) 쓰고 35개열로 학습 및 테스트 결과 951개 0.11가중치 
# rfecv 34개열 954개 0.11
# rfecv 34개열 1255 0.1
# rfecv 34개열 0.105 1154
# 좌표 변수 추가하고 토멕링크 999

# model suffix 1.2 champ temp 1.1 웨이트 주고 0.11 학습 결과 921개
# 1.1 1.05 주고 991 개 0 1 60 근데 노멀임.. 흠
# workorder 만 1.1 1147개
# all normal 설정하고 얘가 임포턴스 15 , 결과 1298개.. 0.11
# all normal 0.12 1073 0.21242
# all normal 0.125 1163 ? 0.9107
# all normal 0.13 972 0.9149 0.19595959
# cat auto_class_weights Balanced 3430
# cat auto_class_weights SqrtBalanced 267
# cat boost_from_average=True, 1471 0.89 (all number)
# cat all normal 0.11 1298
# cat iter 1500 972?
# cat iter 1200 920
# cat iter 1100 1200
# cat iter 1050 1078
# cat 0.11 xgb 0.14 lgbm 0.12 797
# category 설정 후 xgb enable 0.12 , cat 0.11 , lgbm 0.12 
# lgbm category 피쳐 설정
# hard cat 0.11 0.09 xgb 0.09 lgbm 2,1,1 1202
# soft cat 0.11 0.09 xgb 0.09 lgbm 2,1,1 594
# cate lgbm 0.09 1138
# cate lgbm 0.10 1025
# cate xgb 0.09 1117
# cate xgb 0.10 1075
# cate xgb 0.07 lgbm (no cate 설정) 0.07 cat 0.11 800
# cate xgb 0.06 lgbm (no cate 설정) 0.06 cat 0.11 958
# , eval_set=[(valid_data, valid_labels )] t설정하면 개수가 달라짐 (많아짐) 0.12 1115 0.125 1190 개 0.13 939 0.127 967 0.115 1108
# xgb 0.09 cat hard 1000
# 좌표차이 열 빼고 압력 차이, volume 차이열 만 추가해서 110 개열 1361  0.12 921 0.115 1165 0.125 977

# shift 처리 후
# 0.10 1296 개 cat
# 0.07 1130 lgbm  
# 0.07 1209 ?? xgb 인가
#0.07 cat 2292
# all 0.07 cat xgb lgbm hard voting 1126
# all 0.07 cat xgb soft hard voting  906
# cat 0.07 eval set early stopping 100 4321
# cat 0.11 eval set early stopping 100 1466 
# cat 0.12 eval set early stopping 100 1209
# cat 0.125 eval set early stopping 100 1011

# 이상치 제거 (999.9)
# cat 0.125 eval set early stopping 100 이상치 제거(999.9) 1035 0.1888
# cat 0.125 859 
# cat 0.11 그냥 버전 1119
# xgb 0.07 1228
# lgbm 0.07 1080  # category 설정해도 똑같음
# cat 0.11 xgb lgb 0.07 soft 675
# cat 0.11 xgb lgb 0.07 hard 762
# cat 0.07 xgb lgb 0.07 hard 1079
# xgb 0.08 964
# 1160 knn weigth 에 distance, 나머지는 안줌
# knn weight distance, metric cosine 14??
# knn weight distance, metric cosine 이웃6 1366

# knn 이웃 6 나머지 안건듬 1115 178
# knn 이웃 7 나머지 안건듬 1038 171
# knn 이웃 8 나머지 안건듬 1060 174
# knn 이웃 9 나머지 안건듬 1035 172

# 7 맨해튼 1038


In [861]:
df_sub = pd.read_csv("sub.csv")
df_sub["target"] = test_pred
# df_sub['target'] = df_sub['target'].map({0: 'AbNormal', 1: 'Normal'})

print((df_sub['target'] == 'AbNormal').sum())

1035


### 제출 파일 작성


In [759]:
# 제출 파일 저장
df_sub.to_csv("submission_neighbor15_1010.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
